In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset

In [ ]:
#cargamos el dataset completo
dataset = load_dataset("FredZhang7/all-scam-spam")
print(dataset['train'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'is_spam'],
    num_rows: 42619
})


In [ ]:
df_extended = pd.DataFrame(dataset['train'])
df_extended

,text,is_spam
0,Re: New Sequences Window\n\n Date: W...,0
1,[zzzzteana] RE: Alexander\n\nMartin A posted:\...,0
2,[zzzzteana] Moscow bomber\n\nMan Threatens Exp...,0
3,[IRR] Klez: The Virus That Won't Die\n\nKlez:...,0
4,Re: [zzzzteana] Nothing like mama used to make...,0
...,...,...
42614,Какой у тебя любимый цвет? Фантастический един...,1
42615,"Что ты сегодня ел, Чудо-печенье",1
42616,Что ты думаешь о спорте? Секретный ключ,1
42617,"Какой твой любимый вид спорта, Волшебный ковер",1


In [ ]:
df = df_extended.sample(n=10655, random_state=196134)
df

,text,is_spam
37790,No. She's currently in scotland for that.,0
12273,timesheets\n\nhello all :\ni am afraid i did n...,0
15159,vp & director count for the research group\n\n...,0
14437,"re : mr . sud\n\nvince ,\nit never hurts ( or ...",0
7386,hpl expense contracts\n\nfyi . . . . . . . . c...,0
...,...,...
36826,"Alright, I'll head out in a few minutes, text ...",0
28801,full time analysts opp .\n\nthis candidate is ...,0
561,"Since we were on the subject...\n\n\nGays, gay...",0
15536,special report ! tivo : now or never ?\n\nin t...,1


In [ ]:
#feature independiente
X=list(df['text'])
#feature dependiente
#La ventaja del dataset, es que ya viene binario (no ocupamos utilizar la funcioon pd.get_dummies)
y=list(df['is_spam'])

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the text
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Convert the encodings to a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

# Load the pre-trained DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy()
)

# Train the model
model.fit(
    train_dataset.shuffle(1000).batch(8),
    epochs=2,
    batch_size=8,
    validation_data=test_dataset.batch(16)
)

# Save the model
model.save_pretrained('./saved_model/')
# Save tokenizer
tokenizer.save_pretrained('./saved_model')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/2
1066/1066 [==============================] - 549s 493ms/step - loss: 0.1444 - sparse_categorical_accuracy: 0.9440 - val_loss: 0.0792 - val_sparse_categorical_accuracy: 0.9714
Epoch 2/2
1066/1066 [==============================] - 525s 493ms/step - loss: 0.0416 - sparse_categorical_accuracy: 0.9855 - val_loss: 0.0751 - val_sparse_categorical_accuracy: 0.9747


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
def predict_spam(text, tokenizer, model):
    #tokenizamos el texto
    inputs = tokenizer.encode_plus(text, return_tensors="tf", truncation=True, padding=True)

    #predecimos
    predictions = model.predict(inputs.data)[0]

    #el output tiene dos probabilidades; una para "not spam" y una para "spam"
    #usamos `tf.nn.softmax` para convertir los logits a probabilidades
    probs = tf.nn.softmax(predictions, axis=1)

    #cu'al tiene mas probabilidad?
    is_spam = tf.argmax(probs, axis=1).numpy()[0]  # 1 if spam, 0 if not spam

    #definimos tipo
    class_label = "spam" if is_spam else "ham"

    return class_label, probs.numpy()

In [ ]:
#ejemplo
text_to_predict =  "salaan ka waran Ma jeclaan lahayd inaad beri ila qaxweyso?"
label, probabilities = predict_spam(text_to_predict, tokenizer, model)

print(f"The text is classified as: {label}")
print(f"Probability distribution (ham, spam): {probabilities}")

1/1 [==============================] - 2s 2s/step
The text is classified as: ham
Probability distribution (ham, spam): [[0.9359722  0.06402782]]


In [ ]:
!pip install --upgrade huggingface_hub